- Thành viên nhóm:
    - 21120205 Nguyễn Tạ Bảo
    - 21120511 Lê Nguyễn

In [1]:
!pip install chainlit scikit-learn pandas joblib

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import chainlit as cl
from io import StringIO
import threading
import joblib


# 1 - Đọc dữ liệu

In [3]:
train_data = pd.read_csv ('./train.csv')
test_data = pd.read_csv ('./val.csv')

## 1.3 -  5 dòng đầu tiên của dữ liệu train và test

In [4]:
train_data.head(5)

,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,split
0,0,0,christmas tree farm pictures,NaN,ham,0.038415
1,1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,0.696509
2,2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,0.587792
3,3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,-0.055438
4,5,5,mcmullen gas for 11 / 99,"jackie ,\nsince the inlet to 3 river plant is ...",ham,-0.419658


In [5]:
test_data.head(5)

,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,split
0,23,23,miscellaneous,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,-0.351998
1,24,24,re : purge of old contract _ event _ status,fyi - what do you all think ?\n- - - - - - - -...,ham,0.257704
2,32,32,valero 8018 and 1394,it is my understanding the outages valero incu...,ham,0.091200
3,37,37,01 / 00 natural gas nomination,enron methanol company nominates the following...,ham,-1.745133
4,43,43,re : misc . questions,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,-1.911987


# 2 - Tiền xử lý dữ liệu 

In [6]:
# Thay thế các giá trị NaN trong cột Subject bằng chuỗi rỗng
train_data['Subject'] = train_data['Subject'].fillna ('')
test_data['Subject'] = test_data['Subject'].fillna ('')

train_data['Message'] = train_data['Message'].fillna ('')
test_data['Message'] = test_data['Message'].fillna ('')

In [7]:
# Gộp cột Subject và Message lại thành một cột duy nhất là Text 
train_data['Text'] = train_data['Subject'] + ' ' + train_data['Message']
test_data['Text'] = test_data['Subject'] + ' ' + test_data['Message']

In [8]:
# Chia ra tập huấn luyện và tập kiểm thử 
X_train = train_data['Text']
Y_train = train_data['Spam/Ham']

X_test = test_data['Text']
Y_test = test_data['Spam/Ham']

# 3 - Xây dựng mô hình 

In [9]:
# Trích xuất đặc trưng bằng TF-IDF
tfidf = TfidfVectorizer(max_features=5000) # giới hạn số lượng từ vựng tối đa là 5000
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [10]:
# Huấn luyện mô hình Naive Bayes
model = MultinomialNB()
model.fit(X_train_tfidf, Y_train)


MultinomialNB()

In [11]:
# Save the model and vectorizer
joblib.dump(model, 'spam_classifier_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [12]:
# Dự đoán nhãn của tập kiểm thử
Y_pred = model.predict(X_test_tfidf)

In [13]:
# Đánh giá mô hình bằng các độ đo Accuracy, Precision, Recall, F1
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred , pos_label='spam')
recall = recall_score(Y_test, Y_pred, pos_label='spam')
f1 = f1_score(Y_test, Y_pred , pos_label='spam')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.9818417639429312
Precision: 0.9753943217665615
Recall: 0.9891234804862444
F1 Score: 0.9822109275730623


# 4 - Thử nghiệm thực tế

In [14]:
def preprocess_text(text, vectorizer):
    # Giả sử sử dụng cùng vectorizer (TF-IDF)
    return vectorizer.transform([text])

def predict_email(model, vectorizer):
    subject = input("Nhập tiêu đề email: ")
    message = input("Nhập nội dung email: ")

    # Ghép subject và message
    email_text = subject + " " + message
    
    print(f"Subject: {subject}\nMessage: {message}")

    # Tiền xử lý giống như đã làm với tập dữ liệu ban đầu
    email_vectorized = preprocess_text(email_text, vectorizer)

    # Dự đoán
    prediction = model.predict(email_vectorized)

    # Trả về kết quả
    print("Dự đoán: ", "Spam" if prediction[0] == 'spam' else "Ham")

# Ví dụ sử dụng:
predict_email(model, tfidf)  # model là mô hình đã huấn luyện, tfidf là vectorizer


Subject: Hello who are you
Message: I'm spam email
Dự đoán:  Spam


In [15]:
def evaluate_on_csv(model, vectorizer, file_path):
    # Đọc file CSV
    data = pd.read_csv(file_path)
    # Thay thế NaN bằng chuỗi rỗng
    data['Subject'] = data['Subject'].fillna('')
    data['Message'] = data['Message'].fillna('')
    
    # Kết hợp 'subject' và 'message' thành một chuỗi
    data['Text'] = data['Subject'] + " " + data['Message']

    # Tiền xử lý dữ liệu
    X = vectorizer.transform(data['Text'])  # Tiền xử lý giống với quá trình huấn luyện
    y_true = data['Spam/Ham']  # Nhãn thực tế (spam hoặc ham)

    # Dự đoán
    y_pred = model.predict(X)

    # Đánh giá kết quả
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label='spam')
    recall = recall_score(y_true, y_pred, pos_label='spam')
    f1 = f1_score(y_true, y_pred, pos_label='spam')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

# Ví dụ sử dụng:
evaluate_on_csv(model, tfidf, "val.csv")  # model là mô hình đã huấn luyện, tfidf là vectorizer


Accuracy: 0.9818417639429312
Precision: 0.9753943217665615
Recall: 0.9891234804862444
F1 Score: 0.9822109275730623
